In [1]:
import sys
import eastern
import pytz
import numpy as np
import pandas as pd
import shinybroker as sb
from datetime import timedelta

In [2]:
asset_symbol = "MSFT"

In [3]:
############ Risk Tolerance Calculation
asset_data_vix = sb.fetch_historical_data(
    contract=sb.Contract({
        'symbol': 'VIX',  # or 'VX' for VIX futures
        'secType': 'IND',  # Futures contract type
        'exchange': 'CBOE',  # CBOE Futures Exchange for VIX futures
        'currency': 'USD',
    }),
    barSizeSetting='1 day',
    durationStr='1 Y'
)
print(asset_data_vix)
# Extract the DataFrame from the result
df = asset_data_vix['hst_dta']

# Ensure timestamp is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Create a new DataFrame with just the date and scaled high value
risktol_df = pd.DataFrame({
    'Timestamp': df['timestamp'].dt.date,
    'risk_tolerance': df['high'] * 0.01
})

# Print header
print("Timestamp   risk_tolerance")

# Print each row
for _, row in risktol_df.iterrows():
    print(f"{row['Timestamp']}  {row['risk_tolerance']:.4f}")


{'startDateStr': '20240420 14:03:52 US/Central', 'endDateStr': '20250420 14:03:52 US/Central', 'hst_dta':       timestamp   open   high    low  close  volume  wap  barCount
0    2024-04-18  17.91  18.37  17.21  18.00       0  0.0      1989
1    2024-04-19  21.33  21.36  18.17  18.71       0  0.0      2231
2    2024-04-22  18.59  18.72  16.69  16.94       0  0.0      1607
3    2024-04-23  16.72  16.76  15.69  15.69       0  0.0      1290
4    2024-04-24  15.76  16.38  15.58  15.97       0  0.0      1451
..          ...    ...    ...    ...    ...     ...  ...       ...
246  2025-04-11  40.80  46.12  36.85  37.56       0  0.0      2880
247  2025-04-14  34.76  35.17  29.75  30.89       0  0.0      2539
248  2025-04-15  30.01  31.45  28.29  30.12       0  0.0      2561
249  2025-04-16  33.24  34.96  29.48  32.64       0  0.0      2520
250  2025-04-17  30.79  32.55  29.57  29.65       0  0.0      2430

[251 rows x 8 columns]}
Timestamp   risk_tolerance
2024-04-18  0.1837
2024-04-19  0.2136


In [4]:
############ Asset related
asset_ask_data = sb.fetch_historical_data(
    contract=sb.Contract({
        'symbol': asset_symbol,
        'secType': 'STK',
        'exchange': 'SMART',
        'currency': 'USD',
    }),
    barSizeSetting='30 secs',
    durationStr='1 D',
    whatToShow = 'ASK'
)
asset_bid_data = sb.fetch_historical_data(
    contract=sb.Contract({
        'symbol': asset_symbol,
        'secType': 'STK',
        'exchange': 'SMART',
        'currency': 'USD',
    }),
    barSizeSetting='30 secs',
    durationStr='1 D',
    whatToShow = 'BID'
)

ask_df = asset_ask_data['hst_dta'][['timestamp', 'low']].rename(columns={'low': 'ask_low'})
bid_df = asset_bid_data['hst_dta'][['timestamp', 'high']].rename(columns={'high': 'bid_high'})

merged_df = ask_df.merge(bid_df, on='timestamp')
merged_df['mid_price'] = (merged_df['ask_low'] + merged_df['bid_high']) / 2
pd.set_option('display.max_rows', None)
print(merged_df)

############ Set initial parameters
#Rolling Std of Mid-Price Returns
merged_df['log_return'] = np.log(merged_df['mid_price'] / merged_df['mid_price'].shift(1))
window = 30  # Example: 60 periods
merged_df['rolling_volatility'] = merged_df['log_return'].rolling(window=window).std()
merged_df.dropna(subset=['rolling_volatility'], inplace=True)
pd.set_option('display.max_rows', None)
print(merged_df)

              timestamp  ask_low  bid_high  mid_price
0   2025-04-17 09:30:00   373.46    374.11    373.785
1   2025-04-17 09:30:30   373.74    374.02    373.880
2   2025-04-17 09:31:00   373.68    374.28    373.980
3   2025-04-17 09:31:30   373.28    373.51    373.395
4   2025-04-17 09:32:00   372.70    373.24    372.970
5   2025-04-17 09:32:30   372.89    373.10    372.995
6   2025-04-17 09:33:00   372.90    373.21    373.055
7   2025-04-17 09:33:30   372.59    373.21    372.900
8   2025-04-17 09:34:00   372.71    372.92    372.815
9   2025-04-17 09:34:30   372.19    372.67    372.430
10  2025-04-17 09:35:00   372.27    372.52    372.395
11  2025-04-17 09:35:30   372.43    372.61    372.520
12  2025-04-17 09:36:00   372.35    372.67    372.510
13  2025-04-17 09:36:30   372.27    372.68    372.475
14  2025-04-17 09:37:00   372.26    372.56    372.410
15  2025-04-17 09:37:30   371.86    372.13    371.995
16  2025-04-17 09:38:00   371.60    372.00    371.800
17  2025-04-17 09:38:30   37

In [10]:
multiplier = 0
def process_trades(start_time, merged_df, max_attempts=100, cutoff_time='16:00:00'):
    attempt = 0
    result_log = []
    spread = 0  # Initially 0, it will be set below
    stocks_held = 0  # Initialize stocks held
    portfolio_value = 0  # Initialize portfolio value

    # Extract initial data at 09:45:00
    row = merged_df[merged_df['timestamp'] == start_time]
    if row.empty:
        raise ValueError(f"No data found for initial quoting time: {start_time.time()}")

    ask_low = row.iloc[0]['ask_low']
    bid_high = row.iloc[0]['bid_high']
    mid_price = row.iloc[0]['mid_price']

    # Set initial spread to 50% of the market width
    market_width = abs(bid_high - ask_low)
    spread = multiplier * market_width

    # Convert cutoff_time to datetime
    cutoff_time = pd.to_datetime(cutoff_time).time()

    while attempt < max_attempts:
        # Get the row for current start time
        row = merged_df[merged_df['timestamp'] == start_time]
        if row.empty:
            print(f"No data found for start time {start_time.time()}. Skipping this slot.")
            attempt += 1
            continue

        # Check if we've passed the cutoff time and should abort
        if start_time.time() >= cutoff_time:
            print(f"❌ Cutoff time {cutoff_time} reached. Aborting further trades for the day.")
            result_log.append({
                "start_time": start_time.time().strftime('%H:%M:%S'),
                "spread": round(spread, 4),
                "side": "none",
                "order_filled": "No",
                "order_filled_time": None,
                "quote_duration": "N/A",
                "bid_quote": round(mid_price - spread / 2, 4),
                "ask_quote": round(mid_price + spread / 2, 4),
                "filled_price": "N/A",
                "stocks_held": stocks_held,  # Add stocks_held to log
                "portfolio_value": portfolio_value  # Add portfolio_value to log
            })
            break

        mid_price = row.iloc[0]['mid_price']
        #print_quotes(start_time, mid_price, spread)

        # Simulate trade and check if order is filled
        side, filled, fill_time, filled_price = simulate_trade(mid_price, spread, start_time, merged_df)

        if filled:
            # Calculate time duration that the quote was active
            duration = fill_time - start_time
            result_log.append({
                "start_time": start_time.time().strftime('%H:%M:%S'),
                "spread": round(spread, 4),
                "side": side,
                "order_filled": "Yes",
                "order_filled_time": fill_time.strftime('%H:%M:%S'),
                "quote_duration": str(duration),
                "bid_quote": round(mid_price - spread / 2, 4),
                "ask_quote": round(mid_price + spread / 2, 4),
                "filled_price": filled_price,
                "stocks_held": stocks_held,  # Add stocks_held to log
                "portfolio_value": portfolio_value  # Add portfolio_value to log
            })

            # Update stocks_held: Add 10 for long or subtract 10 for short
            stocks_held += 10 if side == "long" else -10

            # Update portfolio value
            portfolio_value = stocks_held * filled_price

            # If order is filled, increase spread by * 2
            spread = spread * 2
            start_time = fill_time + timedelta(seconds=30)  # Next start time is 30s after the fill time
            #print(f"🔁 Order filled at {fill_time.strftime('%H:%M:%S')}. Increasing spread to {spread:.4f}. Starting next cycle at {start_time.strftime('%H:%M:%S')}...\n")
            attempt += 1
        else:
            result_log.append({
                "start_time": start_time.time().strftime('%H:%M:%S'),
                "spread": round(spread, 4),
                "side": "none",
                "order_filled": "No",
                "order_filled_time": None,
                "quote_duration": "N/A",
                "bid_quote": round(mid_price - spread / 2, 4),
                "ask_quote": round(mid_price + spread / 2, 4),
                "filled_price": "N/A",
                "stocks_held": stocks_held,  # Add stocks_held to log
                "portfolio_value": portfolio_value  # Add portfolio_value to log
            })

            # Ensure there are at least 120 rows after current start time
            future_rows = merged_df[merged_df['timestamp'] > start_time]
            if len(future_rows) < 120:
                print("Not enough rows available for the next cycle. Stopping.")
                break

            # Get the 120th row
            new_start_time = future_rows.head(120).iloc[119]['timestamp']  # 120th row (index 119)

            # Check if new start time exceeds cutoff time
            if new_start_time.time() >= cutoff_time:
                print(f"❌ New start time {new_start_time.time()} exceeds cutoff time. Stopping.")
                break

            start_time = new_start_time
            spread = spread / 2  # Decrease spread by 1/2
            #print(f"🔁 No order filled. Updating start time to {start_time.strftime('%H:%M:%S')} and reducing spread to {spread:.4f}.\n")

        # Ensure we loop until max_attempts
        if attempt >= max_attempts:
            break

    # Print result table with proper formatting
    print("\n| start_time | spread | bid_quote | ask_quote | side  | order_filled  | order_filled_time | filled_price | quote_duration  | stocks_held | portfolio_value |")
    print("|------------|--------|-----------|-----------|-------|---------------|-------------------|--------------|-----------------|-------------|-----------------|")
    for entry in result_log:
        print(f"| {entry['start_time']:<10} | {entry['spread']:<6} | {entry['bid_quote']:<9} | {entry['ask_quote']:<9} | {entry['side']:<5} | {entry['order_filled']:<13} | {entry['order_filled_time'] if entry['order_filled_time'] else 'N/A':<17} | {entry['filled_price'] if entry['filled_price'] != 'N/A' else 'N/A':<12} | {entry['quote_duration']:<15} | {entry['stocks_held']:<12} | {entry['portfolio_value']:<16} |")

# === Initial Setup ===
start_time = pd.to_datetime("2025-04-17 09:45:00")  # Starting time for the first quote

# Run the process
process_trades(start_time, merged_df)

NameError: name 'simulate_trade' is not defined